## Import library yang digunakan

In [1]:
import time
import operator as optr
from collections import Counter
import string
import re

## Metode yang digunakan untuk preprocessing

In [2]:
def removePunct(sentence):
    sentence=sentence.lower()                                   #Case Folding, mengubah semua huruf menjadi huruf kecil
    #sentence=sentence.replace('<s>', '').replace('</s>', '')
    sentence=re.sub(r"\d+","", sentence)                        #Filtering, menghapus angka
    for punct in string.punctuation:
        sentence=sentence.replace(punct,"")                     #Filtering, menghapus tanda baca
    return " ".join(sentence.split())                           #Tokenizing, mengubah teks menjadi token + mengubah list menjadi string

## Menentukan misspelled word

In [3]:
#Membaca file kamus
with open('./INDONESIA.txt', 'r', encoding='utf8', errors='ignore') as f:
    wordsFile = f.readlines() # list of valid urdu words, dictionary
wordsList=list()
for word in wordsFile:
    word=word.lower().replace("-","")
    wordsList.append(word[:-1])
#membaca file uji
def readFile(file):
    word=[]
    inputFile=open(file,'r', encoding="utf8")
    for line in inputFile:
        line=removePunct(line)
        line=line.strip().split()
        for words in line:
            word.append(words.strip())
    inputFile.close()
    return word

errLineWords = readFile('Dokumen 1_Uji.txt')
print(len(errLineWords))
correctLineWords = readFile('Dokumen 1_Asli.txt')
misspelledWords=[]
correctSpelledWords=[]
misspelledPrevious=dict()
misspelledNext=dict()
print('Word Index\tWrong word\tCorrect word\tPrevious word\tNext word')
misspell_count=0
no_of_words = -1
if (len(correctLineWords) == len(errLineWords)):
    no_of_words = len(errLineWords) #or len(correctLineWords) # as no of words are equal
else:
    print ('Error line and correct line have word count difference..Abort!!')
for word_idx in range(no_of_words):
    #Fungsi untuk membandingkan kata pada data uji dengan data asli dan pada kamus
    if errLineWords[word_idx] != correctLineWords[word_idx] and errLineWords[word_idx] not in wordsList and errLineWords[word_idx] != '' and correctLineWords[word_idx] != '':
        #print(errLineWords[word_idx], '====', correctLineWords[word_idx])
        misspelledWords.append(errLineWords[word_idx])
        correctSpelledWords.append(correctLineWords[word_idx])
        #menentukan kata sebelum data uji (Wi-1)
        misspelledPrevious[errLineWords[word_idx]] = errLineWords[word_idx-1]
        #menentukan kata setelah data uji (Wi+1)
        misspelledNext[errLineWords[word_idx]] = errLineWords[word_idx+ 1]
        print(misspell_count, '\t\t',errLineWords[word_idx], '\t\t', correctLineWords[word_idx], '\t\t', misspelledPrevious[errLineWords[word_idx]], '\t\t', misspelledNext[errLineWords[word_idx]])
        misspell_count+=1

#Jumlah kata error/misspelled word yang dideteksi
print('\nTotal misspelled word count', misspell_count)

131
Word Index	Wrong word	Correct word	Previous word	Next word
0 		 penelitina 		 penelitian 		 berdasarkan 		 yang
1 		 ksalahan 		 kesalahan 		 bahwa 		 ejaan
2 		 penggunan 		 penggunaan 		 kesalahan 		 huruf
3 		 yng 		 yang 		 baca 		 meliputi
4 		 krangnya 		 kurangnya 		 karena 		 pemahaman

Total misspelled word count 5


## Menemukan kandidat untuk semua ejaan kata salah

In [4]:
# Menemukan Kandidat kata dengan menggunakan jenis DLD, insertion, deletion, substitution, dan transposition
def makeCandidates(word):
    candidates=list()
    charset='abcdefghijklmnopqrstuvwxyz' # alphabet charset
    for char in charset:
        #insertion candidates
        for i in range(len(word)+1):
            candidates.append(word[0:i]+char+word[i:]) 
        #substitution candidates
        for i in range(len(word)):
            candidates.append(word[0:i]+char+word[i+1:])
    #deletion candidates
    for i in range(len(word)):
        candidates.append(word[0:i]+word[i+1:])
    #transpose candidates
    if(len(word)>1):
        for i in range(len(word)-1):
            candidates.append(word[0:i]+word[i+1]+word[i]+word[i+2:])
    return candidates

In [5]:
# Semua kandidat untuk misspelled word
def getAllCandidates():
    candidatesFirstEdit=list()
    candidatesSecondEdit=list()
    for errWord in misspelledWords:
        candidatesFirstEdit+=makeCandidates(errWord)
    for firstEditCandidate in candidatesFirstEdit:
        candidatesSecondEdit+=makeCandidates(firstEditCandidate)
    candidates=set(candidatesFirstEdit).union(set(candidatesSecondEdit)) # menghapus kata duplikat
    candidates=set(wordsList).intersection(candidates) # menghapus kata invalid berdasarkan kata dikamus
    return candidates
candidates=getAllCandidates()

## Menghitung nilai minimum dengan DLD misspelled word dari kandidat kata 

In [6]:
def calculateMinimumEditDistance(str1, str2):
    #str1: ejaan kata salah, str2: candidateWord
    #insertion, deletion, substitution costs are 1
    ic, dc, sc = 1, 1, 1
    n, m=len(str1), len(str2)
    MED_DP=[[0 for x in range(m + 1)] for x in range(n + 1)]  
    for i in range(1,n+1):
        MED_DP[i][0]=MED_DP[i-1][0]+dc
    for i in range(1,m+1):
        MED_DP[0][i]=MED_DP[0][i-1]+dc  
    for i in range(1,n+1):
        for j in range(1,m+1):
            if (i>1 and j>1 and (str1[i-1]==str2[j-2]) and (str1[i-2]==str2[j-1])):
                MED_DP[i][j]=MED_DP[i-2][j-2]+sc
            elif(str1[i-1]==str2[j-1]):
                MED_DP[i][j]=min([MED_DP[i-1][j]+dc,MED_DP[i-1][j-1]+0,MED_DP[i][j-1]+ic])
            else:
                MED_DP[i][j]=min([MED_DP[i-1][j]+dc,MED_DP[i-1][j-1]+sc,MED_DP[i][j-1]+ic])
    #print(MED_DP)
    return MED_DP[n][m]

## Mendapatkan kandidat kata dengan memberikan edit distance

In [7]:
def getCandidateWords(err_word, med):
    candidateWords=list()
    for word in candidates:
        MED=calculateMinimumEditDistance(err_word, word)
        if MED <= med:
            candidateWords.append(word)
    return candidateWords

## Mendapatkan tokens korpus 

In [8]:
def getTrainingSetTokens():
    with open('./Monolingual_corpus_Indonesian.txt', 'r', encoding='utf8', errors='ignore') as f:
        tokens=[]
        for line in f.readlines(): # training set
            line=removePunct(line)
            tokens+=line.split()
        print(len(tokens))
    return tokens

## Fungsi pembuatan NGram 

In [9]:
def makeNGram(n, tokens):
    tokenlen=len(tokens)
    nGramList=[]
    for idx, token in enumerate(tokens):
        singleNGramList=[]
        for i in range(n):
            if(idx+n<=tokenlen):
                singleNGramList.append(tokens[idx+i])
                #print(single_ngramlist)
        if(len(singleNGramList)==n):
            nGramList.append(tuple(singleNGramList))
            #print(single_ngramlist)
            #print(nGramList)
    return nGramList

# Menghitung kemunculan unigrams, bigrams dan trigrams

In [10]:
def calculateCounts():
    tokens=getTrainingSetTokens()
    # fungsi membuat n-gram
    unigramList=makeNGram(1, tokens)
    bigramList=makeNGram(2, tokens)
    trigramList=makeNGram(3, tokens)
    #fungsi menghitung jumlah kemunculan n-gram
    unigramCount=Counter(unigramList) #aggregate all unigrams
    bigramCount=Counter(bigramList) #aggregate all bigrams
    trigramCount=Counter(trigramList)
    return unigramCount, bigramCount, trigramCount
unigramCount, bigramCount, trigramCount = calculateCounts()

1462186


## Menghitung probabilitas dan skor kandidat kata

In [11]:
def calculateProbability(err_word, med):
    #uniLambda = 0.15
    biLambda = 0.4
    triLambda = 0.6
    V = len(unigramCount)
    print(V)
    N = sum(unigramCount.values())
    candidateWordProbabilityDict=dict()
    candidateWords = getCandidateWords(err_word, med)
    #V = len(candidateWords)
    prev_word = misspelledPrevious[err_word]
    next_word = misspelledNext[err_word]
    for word in candidateWords:       
        #unigramProbability=(unigramCount[(word,)]+0.01)/(len(unigramCount)+(0.01*N))
        #if bigramCount[(prev_word, word)]!=0:
        #
        bigramProbability=(bigramCount[(prev_word,word)]+0.01)/(unigramCount[(prev_word),]+(0.01*V))
        #else:
        #    bigramProbability=0
        #if trigramCount[(prev_word, word, next_word)] != 0:
        trigramProbability=(trigramCount[(prev_word,word,next_word)]+0.01)/(bigramCount[(prev_word),word]+(0.01*V))
        #else:
        #    trigramProbability=0
        candidateWordProbability=bigramProbability*biLambda+trigramProbability*triLambda
        #print("kata: ", word, "\t\t", candidateWordProbability)
        if candidateWordProbability != 0:
            candidateWordProbabilityDict[word] = candidateWordProbability
    return candidateWordProbabilityDict

### Menemukan probabilitas berdasarkan edit distance yang diberikan dan peringkat terhadap candidat yang mendekati kemungkinan menjadi koreksi kata

In [12]:
def getWordProbabilities(err_word, correct_word, med, top=10):
    predictedWordsProbabilityDict = calculateProbability(err_word, med)
    data =  sorted(predictedWordsProbabilityDict.items(), key=optr.itemgetter(1), reverse=True)[0:top]
    foundIndex = -1
    for idx, candidate_word in enumerate(data):
        if correct_word in candidate_word[0]:
            foundIndex = idx
            break
    return data, foundIndex


# Final Report

In [15]:
%%time
found_idx=list()

for idx, word in enumerate(misspelledWords):
    print('Word index:', idx, '\t\t', 'False word:', word, '\t', 'True word:', correctSpelledWords[idx], '\t', 'Previous word:', misspelledPrevious[word], '\n')
    med = 1
    predictedWordProbabilityDict, foundIndex = getWordProbabilities(word, correctSpelledWords[idx], med)
    if foundIndex == -1:
        med += 1
        predictedWordProbabilityDict, foundIndex = getWordProbabilities(word, correctSpelledWords[idx], med)
    if foundIndex == -1:
        print ('True word not found in top 10', med, 'edit distance candidate words\n')
    else:
        found_idx.append(idx)
        print ('True word found in top 10 predicted', med, 'edit distance candidates with bigram probability: :%.6f' % predictedWordProbabilityDict[foundIndex][1], '\n')
    print ('Candidate Index\t\tCandidate word\t\t\tProbability')
    for idx1, candidate_word in enumerate(predictedWordProbabilityDict):
        print_str = str(str(idx1) + '\t\t\t' + candidate_word[0]) + '\t\t\t' + str(candidate_word[1])
        if foundIndex==idx1:
            print_str =  '\x1b[0;30;43m'+'\33[1m'+print_str+' ----> TRUE WORD'+'\x1b[0m'
        print (str(print_str))
    print ('----------------------------------------------------------------------------------------------------------------\n')

print('Following are the word indices of words whose correct word was found in top 10 candidates:\n', found_idx, '\n')
print('Total misspelled words whose correct word was found', len(found_idx), '\n')

Word index: 0 		 False word: penelitina 	 True word: penelitian 	 Previous word: berdasarkan 

37310
True word found in top 10 predicted 1 edit distance candidates with bigram probability: :0.005492 

Candidate Index		Candidate word			Probability
0			penelitian			0.0054924691787497665 ----> TRUE WORD
1			penelitinya			2.180312160739958e-05
----------------------------------------------------------------------------------------------------------------

Word index: 1 		 False word: ksalahan 	 True word: kesalahan 	 Previous word: bahwa 

37310
True word found in top 10 predicted 1 edit distance candidates with bigram probability: :0.000191 

Candidate Index		Candidate word			Probability
0			kesalahan			0.0001910779293936624 ----> TRUE WORD
1			kalahan			1.6663289215245e-05
2			kealahan			1.6663289215245e-05
----------------------------------------------------------------------------------------------------------------

Word index: 2 		 False word: penggunan 	 True word: penggunaan 	 Prev